# PyTorch Transforms

## Why transforms
Transforms let you convert raw dataset outputs (PIL images, ndarray) into tensors and perform preprocessing/augmentation in a consistent, composable way. I prefer to keep transforms with the dataset definition so training code stays clean.

In [1]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose, Normalize, RandomHorizontalFlip, RandomCrop
import matplotlib.pyplot as plt

## Simple example
`ToTensor()` converts a PIL Image or NumPy array into a FloatTensor and scales pixel values to [0., 1.]. Use this as the first transform when you need tensors.

In [2]:
ds = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
img, label = ds[0]
print(img.shape, img.min().item(), img.max().item(), label)

100%|██████████| 26.4M/26.4M [00:02<00:00, 12.3MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 211kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.94MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 12.1MB/s]

torch.Size([1, 28, 28]) 0.0 1.0 9


## Target transforms
I often convert labels to one-hot when experimenting with custom losses or multi-label experiments. `Lambda` is a concise way to do this inline.

In [3]:
target_transform = Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1))
ds_onehot = datasets.FashionMNIST(root="data", train=True, download=False, transform=ToTensor(), target_transform=target_transform)
_, lab = ds_onehot[1]
print(lab.shape, lab.sum().item())

torch.Size([10]) 1.0


## Compose common pipeline
Compose lets you chain transforms; I typically normalize after converting to tensor and apply simple augmentations for training.

In [4]:
train_transforms = Compose([
    RandomCrop(28, padding=4),
    RandomHorizontalFlip(),
    ToTensor(),
    Normalize((0.5,), (0.5,))
])
ds_aug = datasets.FashionMNIST(root="data", train=True, download=False, transform=train_transforms)
img_aug, _ = ds_aug[2]
print(img_aug.shape, img_aug.mean().item(), img_aug.std().item())

torch.Size([1, 28, 28]) -0.7163165211677551 0.3897533416748047


## Quick rules I follow
- Always apply `ToTensor()` before `Normalize()`.
- Use `Compose` for readability.
- Keep heavy augmentations only on training pipeline.
- For debugging set `num_workers=0` in DataLoader to avoid worker-related surprises.